In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

import os
import numpy as np
import music21
from music21 import converter,note,instrument,chord
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM,Activation
from keras.utils import np_utils
import glob

In [17]:
os.chdir('/content/gdrive/My Drive/')
notes = []
for file in glob.glob("music_folder/*.mid"):
    midi = converter.parse(file)
    notesparser = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notesparser = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notesparser = midi.flat.notes
    for element in notesparser:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

n_vocab = len(set(notes))
print(notes)


['A5', 'A3', 'E6', 'E4', 'D6', 'D4', 'E6', 'E4', 'D6', 'D4', 'C6', 'C4', 'D6', 'D4', 'E6', 'E4', 'F6', 'F4', 'G6', 'G4', 'E6', 'E4', 'F6', 'F4', 'E6', 'E4', 'D6', 'E6', 'D4', 'E4', 'D6', 'D4', 'C6', 'C4', 'B-5', 'B-3', 'A2', 'A3', 'E3', 'C#4', 'A3', 'E4', 'C#4', 'A4', 'E4', 'C#5', 'A4', 'E5', 'C#5', 'A4', 'C#4', 'C#5', 'E4', 'E5', 'A4', 'A5', 'C#5', 'C#6', 'E5', 'E6', 'A5', 'C#6', 'E5', 'A5', 'C#5', 'E5', 'A4', 'C#5', 'E4', 'A4', 'C#4', 'E5', 'A4', 'C#5', 'E4', 'A4', 'C#4', 'E4', 'A3', 'C#4', 'A3', 'A3', 'E3', 'A2', 'A5', 'A3', 'E6', 'E4', 'D6', 'D4', 'E6', 'E4', 'D6', 'D4', 'C6', 'C4', 'D6', 'D4', 'E6', 'E4', 'F6', 'F4', 'G6', 'G4', 'E6', 'E4', 'F6', 'F4', 'E6', 'E4', 'D6', 'E6', 'D4', 'E4', 'D6', 'D4', 'C6', 'C4', 'B-5', 'B-3', 'A5', 'A3', 'B-5', 'B-3', 'C6', 'C4', 'D6', 'D4', 'F6', 'F4', 'D6', 'E6', 'D4', 'E4', 'D6', 'D4', 'C6', 'C4', 'B-5', 'B-3', 'A5', 'A3', 'B-5', 'B-3', 'C6', 'C4', 'D6', 'D4', 'E6', 'E4', 'G6', 'G4', 'F6', 'F4', 'D6', 'D4', 'E6', 'E4', 'D6', 'D4', 'C6', 'C4', 'B

In [0]:
 def createinputlstm():
  music_length = 100
  pitchnames = set(item for item in notes)
  note_to_int={note:number for number,note in enumerate(pitchnames)}
  network_input = []
  network_output = []
  # create input sequences and the corresponding outputs
  for i in range(0, len(notes) - music_length):
      sequence_in = notes[i:i + music_length]
      sequence_out = notes[i + music_length]
      network_input.append([note_to_int[char] for char in sequence_in])
      network_output.append(note_to_int[sequence_out])
  n_patterns = len(network_input)
  # reshape the input into a format compatible with LSTM layers
  network_input = np.reshape(network_input, (n_patterns, music_length, 1))
  # normalize input
  n_vocab = len(set(notes))
  network_input = network_input / float(n_vocab)
  network_output = np_utils.to_categorical(np.array(network_output))
  print(np.shape(network_input))
  print(np.shape(network_output))
  return network_input,network_output

In [15]:
network_input,network_output=createinputlstm()

(18319, 100, 1)
(18319, 219)


In [18]:
    model = Sequential()
    model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adagrad')
    model.fit(network_input, network_output, batch_size =20, nb_epoch = 10,  verbose = 5)
    
    model_json=model1.to_json()
    with open("model_.json","w") as json_file:
      json_file.write(model_json)
      
    model1.save_weights("music.h5")
    print("lan")

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/10


KeyboardInterrupt: ignored

In [0]:
model_json=model.to_json()
with open("model_.json","w") as json_file:
  json_file.write(model_json)
      
model.save_weights("music.h5")
print("lan")

lan


In [0]:
modeltest = Sequential()
modeltest.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
modeltest.add(Dropout(0.3))
modeltest.add(LSTM(512, return_sequences=True))
modeltest.add(LSTM(256))
modeltest.add(Dense(256))
modeltest.add(Dense(n_vocab))
modeltest.add(Activation('softmax'))
modeltest.compile(loss='categorical_crossentropy', optimizer='rmsprop')
modeltest.load_weights("music.h5")

In [0]:

int_to_note={number:note for number,note in enumerate(pitchnames)}
#generating 500 notes by taking first 100 notes of the given input
sequence = network_input[0]
pattern=[]
for i in range(sequence.size):
  pattern.append(sequence[i][0])
prediction_output = []

for i in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

NameError: ignored

In [0]:
offset = 0
output_notes = []

    # create note and chord objects based on the values generated by the model
for output in prediction_output:
        # pattern is a chord
  if ('.' in output) or pattern.isdigit():
    notes_in_chord = output.split('.')
    notes = []
    for current_note in notes_in_chord:
      new_note = note.Note(int(current_note))
      new_note.storedInstrument = instrument.Piano()
      notes.append(new_note)
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
        # pattern is a note
  else:
    new_note = note.Note(output)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

  offset += 0.5
print(output_notes)
mf = MIDIFile(1) 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

[<music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.Note B->, <music21.note.N

TypeError: ignored

In [0]:

def fun(a):
  while a<1000:
    a+=1
  return a
    